In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.db.models import Sum, Max, Subquery, Q, F, Avg, Count, Min, Max, Sum, F, Q
from django.db.models import Value, IntegerField
from django.db.models.functions import Coalesce, Greatest, Floor
from account.models import User
from booth.models import Transaction, Participation
from player.models import Player
import random

In [2]:

random_index = {
    'money': [10000,10000,15000,20000,25000,30000,35000,40000,45000,50000],
    'health_score': [20, 40, 40, 60, 60, 80, 80, 100, 100, 120],
    'skill_score': [20, 20, 40, 40, 60, 60, 80, 100, 120, 130],
    'growth_score': [5, 5, 5, 10, 10, 15, 20, 25, 30, 35],
    'relationship_score': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'academic_level': [0, 0, 0, 0, 1, 1, 1, 2, 2, 4]
}
def create_player(u):
    Player.objects.create(
        user=u,
        born_money=random.choice(random_index['money']),
        born_health_score=random.choice(random_index['health_score']),
        born_skill_score=random.choice(random_index['skill_score']),
        born_growth_score=random.choice(random_index['growth_score']),
        born_relationship_score=random.choice(random_index['relationship_score']),
        born_academic_level=random.choice(random_index['academic_level']),
        born_steps=15
    )

In [3]:
# delete all existing players
# Player.objects.all().delete()

In [3]:
students = User.objects.filter(id__gte=290, id__lte=294)
# others = User.objects.filter(id__gt=500)
for u in students:
    create_player(u)
# for u in others:
#     create_player(u)

In [7]:
Player.objects.filter(active=False).delete()

(723, {'booth.Participation': 1, 'booth.Transaction': 2, 'player.Player': 720})